In [1]:
# Importing the libraries needed to process the data

import pandas as pd
import numpy as np

In [2]:
##Reading the data from the file location

data=pd.read_csv('energydata_complete.csv')

In [3]:
##To display the first five rows of the dataset
data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [4]:
# To Check the dataset for missing values in each of the columns
data.isna().sum()

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

In [5]:
data.shape

(19735, 29)

In [6]:
##To Check for duplicated rows
data.duplicated().any()

False

In [7]:
# To Drop the data column before rescalling since it is not a time series problem
## To Drop the light column
data = data.drop(["date","lights"],axis=1)

In [12]:
# To Normalize the dataset using MinMaxScaler

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(data),columns=data.columns)

In [13]:
##Seperate the dataset into attributes and targets, that is the X and y variable

X = scaled_data.drop(["Appliances"],axis=1)
y = scaled_data["Appliances"]

In [11]:
##Splitting the dataset for training and testing 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [14]:
##Creating a simple linear reggression model to predict T6 using T2
x1 = np.array(scaled_data["T2"])
y1 = np.array(scaled_data["T6"])
#We need to reshape the x variable because we have only one predicator
x1 = x1.reshape(-1,1)
x1_train, x1_test, y1_train, y1_test = train_test_split(
    x1, y1, test_size=0.3, random_state=42)
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x1_train,y1_train)
y1_pred = model.predict(x1_test)
from sklearn.metrics import r2_score
r2 = r2_score(y1_test,y1_pred)
round(r2,2)

0.64

In [15]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train,y_train)
y_pred = reg.predict(X_test)
train_score = reg.score(X_train,y_train)
test_score = reg.score(X_test, y_test)
print("Training Score of linear Regression model is {:.3f}".format(train_score))
print("Testing Score of linear Regression model is {:.3f}".format(test_score))

Training Score of linear Regression model is 0.145
Testing Score of linear Regression model is 0.149


In [16]:
##Mean Absolute Error(mae)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test,y_pred)
print("mean absolute error for linear_model:{}".format(round(mae,2)))

mean absolute error for linear_model:0.05


In [17]:
##Residual Sum of Squared(RSS)
rrs = np.sum(np.square(y_test-y_pred))
print("Residual sum of linear_model: {}".format(round(rrs,2)))

Residual sum of linear_model: 45.35


In [18]:
##Root mean error(rme)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
print("Root mean squared error for linear_model: {}".format(round(rmse,3)))

Root mean squared error for linear_model: 0.088


In [19]:
##R_squared(cooefficient of determination)
from sklearn.metrics import r2_score
r_squared = r2_score(y_test,y_pred)
print("R_squared for linear_model {:.2f}".format(r_squared))

R_squared for linear_model 0.15


In [21]:
##to Check the weight of the feactures using linear regression
def weights(model,feat,col_name):
    weights = pd.Series(model.coef_,feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ["Features",col_name]
    weights_df[col_name].round(3)
    return weights_df
df = weights(reg,X_train,"Linear_model_weight")

In [22]:
df.head(2)

,Features,Linear_model_weight
0,RH_2,-0.456698
1,T_out,-0.321860


In [23]:
##Lasso model at alpha=0.001
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.001)
lasso.fit(X_train,y_train)
lasso_pred = lasso.predict(X_test)
train_score = lasso.score(X_train,y_train)
test_score = lasso.score(X_test,y_test)
print("Training score of lasso regression model is {:.3f}".format(train_score))
print("Testing score of lasso regression model is {:.3f}".format(test_score))
print("Number of features for training: {:.3f}".format(np.sum(lasso.coef_!=0)))

Training score of lasso regression model is 0.025
Testing score of lasso regression model is 0.027
Number of features for training: 4.000


In [24]:
## Rmse for lasso model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test,lasso_pred)
rmse = np.sqrt(mse)
print("root mean squared error for lasso model:{}".format(round(rmse,3)))

root mean squared error for lasso model:0.094
